In [ ]:
%load_ext autoreload
%autoreload 2
import os

from dotenv import load_dotenv

load_dotenv("../config/prod.env")

import yaml

with open("../config/config.yaml") as f:
    cfg = yaml.load(f, Loader=yaml.FullLoader)

from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

import pandas as pd
import mlflow


# enter details of your AML workspace
subscription_id = os.getenv("subscription_id")
resource_group = os.getenv("resource_group")
workspace = os.getenv("workspace")

from azure.identity import InteractiveBrowserCredential

credential = InteractiveBrowserCredential()  # DefaultAzureCredential()
# get a handle to the workspace
ml_client = MLClient(credential, subscription_id, resource_group, workspace)

mlflow.set_tracking_uri(ml_client.workspaces.get(cfg["workspace"]).mlflow_tracking_uri)

# Create model from local file

In [ ]:
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes

file_model = Model(
    path=cfg["model"]["path"],
    type=AssetTypes.CUSTOM_MODEL,
    name=cfg["model"]["name"],
    description=cfg["model"]["description"],
)
ml_client.models.create_or_update(file_model)

# Load model from remote and score

In [ ]:
aml_model = ml_client.models.get(name=cfg["model"]["name"], version=1)

In [ ]:
model = mlflow.pyfunc.load_model("models:/<>/latest")

In [ ]:
model.predict(data=...)

# Download model and score

In [ ]:
ml_client.models.download(
    name=cfg["model"]["name"], version=1, download_path=cfg["model"]["local_dev_path"]
)

In [ ]:
import joblib

model = joblib.load(os.path.join(cfg["model"]["local_dev_path"], "lr_model"))

In [ ]:
model.predict(...)